In [1]:
%load_ext autoreload 
%autoreload 1
import os
# os.environ['SNORKELDB'] = 'sqlite:///brat-import.db'
from set_env import set_env
set_env()
from snorkel import SnorkelSession
import snorkel.contrib.brat as brt
from snorkel.parsers import StanfordCoreNLPServer

from snorkel.models import Candidate, Document, candidate_subclass, GoldLabel

session = SnorkelSession()

In [3]:
corenlp_server = StanfordCoreNLPServer(version='3.6.0', split_newline=False, num_threads=1)

In [4]:
brat = brt.Brat(session)

In [4]:
# Only run on documents which have annotations

# from snorkel.parser import TextDocPreprocessor, CorpusParser
# import multiprocessing
# doc_preprocessor = TextDocPreprocessor("brat_data/test_labeled_docs/*.txt", encoding="utf-8")
# corpus_parser = CorpusParser()
# corpus_parser.apply(doc_preprocessor, parralelism=multiprocessing.cpu_count())

In [5]:
input_dir = "brat_data/try_2/"
brat.import_project(input_dir, annotations_only=False, annotator_name='brat', num_threads=1, parser=corenlp_server)

PMC2295252 {'char_start': 6524, 'span': [u'ca'], 'sent_id': 116, 'mention': u'ca. six extra rosette leaves,', 'char_end': 6553, 'idx_span': (4, 9)} Couldn't find sentence


In [6]:
pheno_spans = brat.explore()
print(len(pheno_spans))
print len([pheno_span for pheno_span in pheno_spans if len(pheno_span) > 1])
for p in pheno_spans:
    print p

680
126
[TemporarySpan("chromosome fragmentation in Atmnd1 was suppressed by loss of Atspo11-1", sentence=459551, chars=[62,131], words=[9,18])]
[TemporarySpan("did not affect normal vegetative development but caused fragmentation and missegregation of chromosomes in male and female meiosis, formation of inviable gametes, and sterility", sentence=459549, chars=[15,190], words=[3,28])]
[TemporarySpan("sensitive to salt-induced osmotic stress", sentence=449905, chars=[15,54], words=[4,8])]
[TemporarySpan("root growth was reduced in comparison with wild type on 50 and 100 mM NaCl", sentence=449899, chars=[108,181], words=[24,38])]
[TemporarySpan("sensitive to other monovalent salts, such as KCl and LiCl, and to mannitol", sentence=449903, chars=[26,99], words=[5,19])]
[TemporarySpan("more sensitive to salt stress", sentence=449894, chars=[37,65], words=[8,12])]
[TemporarySpan("modestly salt sensitive", sentence=450019, chars=[276,298], words=[50,52])]
[TemporarySpan("Salt stress sensitivi

In [7]:
from snorkel.models import Document, Sentence, Span

In [8]:
PhenoPair = candidate_subclass('Phenotypes', ['descriptor', 'entity'])

In [21]:
import collections
from sqlalchemy import and_
from snorkel.models import StableLabel, GoldLabel, GoldLabelKey

jaccard_cutoff = 0.3
'''
ak = session.query(GoldLabelKey).filter(GoldLabelKey.name == 'gold').first()
if ak is None:
    ak = GoldLabelKey(name='gold')
    session.add(ak)
else:

session.commit()
    
'''
candidates = session.query(PhenoPair).filter(PhenoPair.split == 0).all()
print(len(candidates), "candidates")
sentence_to_pheno = collections.defaultdict(list)

for pheno_span in pheno_spans:
    # pheno_span is a list of fragments, where each fragment is a temporary span
    if pheno_span:
        sent_id = pheno_span[0].sentence.id
        sentence_to_pheno[sent_id].append(pheno_span)

num = 0
num_matches = 0
num_sentences = 0
num_gold_phenos = 0
num_zero_matches = 0
avg = 0

labeled = {}

for sentence_id, gold_phenos in sentence_to_pheno.iteritems():
    if num%1000==0:print num
    sentence_match = False
    results = session.query(Span, PhenoPair).filter(and_(Span.sentence_id == sentence_id, PhenoPair.descriptor_id == Span.id)).all()
    if (len(results) == 0):
        num_zero_matches += 1
#         print "sentence", sentence_id
#         for gold_pheno in gold_phenos:
#             print(gold_pheno.get_span())
#         print
        
    for gold_pheno_fragments in gold_phenos:
        matched = False
        for span, cand_pheno in results:
            # Build word indice set for each of the gold phenotype, and the candiate phenotype
            cand_words = set()
            gold_words = set()
            # combine both entities (modifier and attribute) into one "phenotype"
            [cand_words.update(xrange(span.get_word_start(), span.get_word_end()+1)) for span in cand_pheno.get_contexts()]
            
            for gold_pheno in gold_pheno_fragments:
                gold_words.update(xrange(gold_pheno.get_word_start(), gold_pheno.get_word_end()+1))
            intersect = gold_words.intersection(cand_words)
            jaccard_score = float(len(intersect)) / len(cand_words.union(gold_words))
            
            val = -1

            #if max(gold_words) - min(gold_words) == len(gold_words)+1 and max(cand_words)-min(cand_words) != len(cand_words)+1:
            #    continue
                
            if jaccard_score > jaccard_cutoff:
                matched = True
                sentence_match = True
                avg += jaccard_score
                num += 1
                val = 1
            '''
            label = session.query(GoldLabel).filter(GoldLabel.key == ak).filter(GoldLabel.candidate == cand_pheno).first()
            if label is None:
                session.add(GoldLabel(candidate=cand_pheno, key=ak, value=val))
            elif label.value == -1 and val == 1:
                label.value = val
            '''

        num_gold_phenos += 1
        if matched:
            num_matches += 1
    if sentence_match:
        num_sentences += 1
        
#session.commit()

print("avg", avg / num)
print("num matches", num_matches)
recall = float(num_matches) / num_gold_phenos
print("num gold phenos", num_gold_phenos)
num_sentences_tagged = len(sentence_to_pheno.keys())
print("num_sentences_tagged by tanya", num_sentences_tagged)
print("sentences", float(num_sentences) / num_sentences_tagged)
print("num sents missed", float(num_zero_matches) / num_sentences_tagged)
print("recall", recall)

(39176, 'candidates')
0
('avg', 0.56404553128811)
('num matches', 534)
('num gold phenos', 680)
('num_sentences_tagged by tanya', 611)
('sentences', 0.7986906710310966)
('num sents missed', 0.10638297872340426)
('recall', 0.7852941176470588)


In [25]:
# each entity/relation type is assigned to a different split
#for i in range(0,1):

print len(candidates)
for i,c in enumerate(candidates):
    #label = session.query(GoldLabel).filter(GoldLabel.key == ak).filter(GoldLabel.candidate == c).first()
    #if label is not None and label.value == 1: print c, label
    print type(c).type, c
    if i > 5:
        break
print
    
sents = list(set([pheno_span[0].sentence.id for pheno_span in pheno_spans if pheno_span]))
candidates = [c for c in candidates if c[0].sentence.id in sents]

# ProteinComponent relations
#candidates = session.query(PhenoPair).filter(PhenoPair.split == 0).all()

0



### this is how we can selectively only look at the labeled sentences for training/lf functions

In [26]:
sents = set([pheno_span[0].sentence.id for pheno_span in pheno_spans if pheno_span])
print len(sents)
c = [p for p in session.query(PhenoPair).filter(PhenoPair.split == 0).all() if p and p[0].sentence.id in sents]
print len(c)

611
0


In [13]:
print len(c)

4083


In [111]:
#IGNORE THIS CELL
import numpy as np

splits = set(np.random.choice(np.arange(0,86), 43, replace=False))
dev = set()
test = set()
print len(splits)
with open('small_data/tair_labels.txt', 'rb') as f:
    lines = set(f.readlines())
    print len(lines)
    for i,line in enumerate(lines):
        line=line.strip()
        if i in splits:
            if line in dev: print line
            dev.add(line)
        else:
            if line in test:print line
            test.add(line)
print i
print len(dev), len(test)

[TemporarySpan("", sentence=459557, chars=[62,131], words=[0,0])]
371416
43
86
85
43 43


## @Tara - this is where I'm creating new splits

In [12]:
import numpy as np

doc_ids = list(set([p[0].sentence.document.id for p in session.query(PhenoPair).filter(PhenoPair.split == 0).all() if p[0].sentence.id in sents]))

In [13]:
np.random.seed(742)
split_size = len(doc_ids)/2
split_1 = np.random.choice(doc_ids, split_size, replace=False)
'''
if session.query(PhenoPair).filter(PhenoPair.split == 1):
    for p in session.query(PhenoPair).filter(PhenoPair.split == 1):
        p.clear()

if session.query(PhenoPair).filter(PhenoPair.split == 2):
    for p in session.query(PhenoPair).filter(PhenoPair.split == 2):
        p.clear()
'''
num_c = 0
for cand in c:
    cand.split = 1
    if cand.get_contexts()[0].sentence.document.id in split_1:
        cand.split = 2
        num_c += 1
session.commit()

print len(c)
print num_c


4083
2014


In [157]:
for p in session.query(PhenoPair).filter(PhenoPair.split == 1):
    print p

Phenotypes(Span("reduced fertility", sentence=439618, chars=[238,254], words=[45,46]), Span("pleiotropic growth", sentence=439618, chars=[139,156], words=[32,33]))
Phenotypes(Span("reduced fertility", sentence=439618, chars=[238,254], words=[45,46]), Span("reduced fertility", sentence=439618, chars=[238,254], words=[45,46]))
Phenotypes(Span("reduced fertility", sentence=439618, chars=[238,254], words=[45,46]), Span("early flowering", sentence=439618, chars=[197,211], words=[38,39]))
Phenotypes(Span("reduced fertility", sentence=439618, chars=[238,254], words=[45,46]), Span("developmental phenotypes", sentence=439618, chars=[162,185], words=[35,36]))
Phenotypes(Span("reduced fertility", sentence=439618, chars=[238,254], words=[45,46]), Span("elongated petioles", sentence=439618, chars=[214,231], words=[41,42]))
Phenotypes(Span("reduced fertility", sentence=439618, chars=[238,254], words=[45,46]), Span("grown in long photoperiod", sentence=439618, chars=[261,285], words=[48,51]))
Phenoty

Phenotypes(Span("quantitative", sentence=440201, chars=[78,89], words=[12,12]), Span("mutants by quantitative RT-PCR 6 h post inoculation with Pst DC3000", sentence=440201, chars=[67,133], words=[10,20]))
Phenotypes(Span("quantitative", sentence=440201, chars=[78,89], words=[12,12]), Span("10mM MgCl2", sentence=440201, chars=[154,163], words=[25,26]))
Phenotypes(Span("expression was analyzed in 4-week-old wild-type", sentence=440201, chars=[5,51], words=[1,6]), Span("mutants by quantitative RT-PCR 6 h post inoculation with Pst DC3000", sentence=440201, chars=[67,133], words=[10,20]))
Phenotypes(Span("expression was analyzed in 4-week-old wild-type", sentence=440201, chars=[5,51], words=[1,6]), Span("10mM MgCl2", sentence=440201, chars=[154,163], words=[25,26]))
Phenotypes(Span("mock treatment", sentence=440201, chars=[138,151], words=[22,23]), Span("mutants by quantitative RT-PCR 6 h post inoculation with Pst DC3000", sentence=440201, chars=[67,133], words=[10,20]))
Phenotypes(Span("mo

Phenotypes(Span("clearly separated", sentence=439963, chars=[4,20], words=[1,2]), Span("PC1 clearly separated pie1 from the rest of the group", sentence=439963, chars=[0,52], words=[0,9]))
Phenotypes(Span("clearly separated", sentence=439963, chars=[4,20], words=[1,2]), Span("PC3 defined the contribution of arp6", sentence=439963, chars=[63,98], words=[12,17]))
Phenotypes(Span("fully expanded", sentence=451270, chars=[3,16], words=[1,2]), Span("found in the vascular tissue", sentence=451270, chars=[42,69], words=[7,11]))
Phenotypes(Span("fully expanded", sentence=451270, chars=[3,16], words=[1,2]), Span("fully expanded leaves GUS staining", sentence=451270, chars=[3,36], words=[1,5]))
Phenotypes(Span("was less intensely stained than the lateral", sentence=451274, chars=[23,65], words=[5,11]), Span("less intensely stained than the lateral veins", sentence=451274, chars=[27,71], words=[6,12]))
Phenotypes(Span("was less intensely stained than the lateral", sentence=451274, chars=[23,65], 

Phenotypes(Span("than longitudinal growth", sentence=425604, chars=[62,85], words=[9,11]), Span("always more sensitive to inhibitors than longitudinal growth", sentence=425604, chars=[26,85], words=[4,11]))
Phenotypes(Span("than longitudinal growth", sentence=425604, chars=[62,85], words=[9,11]), Span("Radial root expansion", sentence=425604, chars=[0,20], words=[0,2]))
Phenotypes(Span("expansion was always more sensitive to", sentence=425604, chars=[12,49], words=[2,7]), Span("always more sensitive to inhibitors than longitudinal growth", sentence=425604, chars=[26,85], words=[4,11]))
Phenotypes(Span("expansion was always more sensitive to", sentence=425604, chars=[12,49], words=[2,7]), Span("Radial root expansion", sentence=425604, chars=[0,20], words=[0,2]))
Phenotypes(Span("were visible by light or", sentence=412056, chars=[156,179], words=[26,30]), Span("more starch granules per chloroplast than mature leaves", sentence=412056, chars=[49,103], words=[7,14]))
Phenotypes(Span("were 

Phenotypes(Span("number of wild-type", sentence=435704, chars=[46,64], words=[8,10]), Span("meristem", sentence=435704, chars=[8,15], words=[1,1]))
Phenotypes(Span("number of wild-type", sentence=435704, chars=[46,64], words=[8,10]), Span("cell number of wild-type plants", sentence=435704, chars=[41,71], words=[7,11]))
Phenotypes(Span("size indicated in", sentence=435704, chars=[17,33], words=[2,4]), Span("meristem", sentence=435704, chars=[8,15], words=[1,1]))
Phenotypes(Span("size indicated in", sentence=435704, chars=[17,33], words=[2,4]), Span("cell number of wild-type plants", sentence=435704, chars=[41,71], words=[7,11]))
Phenotypes(Span("were measured at different growth", sentence=402928, chars=[128,160], words=[21,25]), Span("measured at different growth stages of leaf development", sentence=402928, chars=[133,187], words=[22,29]))
Phenotypes(Span("were measured at different growth", sentence=402928, chars=[128,160], words=[21,25]), Span("chlorophyll b", sentence=402928, chars

Phenotypes(Span("decreased during dark-induced senescence in wild-type", sentence=419515, chars=[53,105], words=[9,14]), Span("decreased during dark-induced senescence in wild-type leaves", sentence=419515, chars=[53,112], words=[9,15]))
Phenotypes(Span("decreased during dark-induced senescence in wild-type", sentence=419515, chars=[53,105], words=[9,14]), Span("consistent with the observed decrease in free IAA content", sentence=419515, chars=[124,180], words=[19,27]))
Phenotypes(Span("content", sentence=419515, chars=[174,180], words=[27,27]), Span("Transcript levels of the auxin-inducible gene", sentence=419515, chars=[0,44], words=[0,5]))
Phenotypes(Span("content", sentence=419515, chars=[174,180], words=[27,27]), Span("decreased during dark-induced senescence in wild-type leaves", sentence=419515, chars=[53,112], words=[9,15]))
Phenotypes(Span("content", sentence=419515, chars=[174,180], words=[27,27]), Span("consistent with the observed decrease in free IAA content", sentence=419

Phenotypes(Span("expression", sentence=411946, chars=[54,63], words=[9,9]), Span("RACK1 genes positively regulate each other", sentence=411946, chars=[9,50], words=[2,7]))
Phenotypes(Span("positively regulate each other", sentence=411946, chars=[21,50], words=[4,7]), Span("RACK1 genes positively regulate each other", sentence=411946, chars=[9,50], words=[2,7]))
Phenotypes(Span("response to", sentence=436334, chars=[215,225], words=[29,30]), Span("http://bar.utoronto.ca/welcome.htm;43]", sentence=436334, chars=[109,146], words=[18,18]))
Phenotypes(Span("response to", sentence=436334, chars=[215,225], words=[29,30]), Span("abiotic/biotic stimuli as well as signal transduction", sentence=436334, chars=[238,290], words=[33,39]))
Phenotypes(Span("classification super", sentence=436334, chars=[79,98], words=[14,15]), Span("http://bar.utoronto.ca/welcome.htm;43]", sentence=436334, chars=[109,146], words=[18,18]))
Phenotypes(Span("classification super", sentence=436334, chars=[79,98], words=[1

Phenotypes(Span("faint bands derived from", sentence=459439, chars=[127,150], words=[20,23]), Span("AtCWINV4 transcripts", sentence=459439, chars=[14,33], words=[3,4]))
Phenotypes(Span("faint bands derived from", sentence=459439, chars=[127,150], words=[20,23]), Span("30 cycles of RT-PCR", sentence=459439, chars=[106,124], words=[15,18]))
Phenotypes(Span("faint bands derived from", sentence=459439, chars=[127,150], words=[20,23]), Span("only visible at 35 cycles", sentence=459439, chars=[171,195], words=[27,31]))
Phenotypes(Span("faint bands derived from", sentence=459439, chars=[127,150], words=[20,23]), Span("readily detected from whole flowers of wild-type plants", sentence=459439, chars=[40,94], words=[6,13]))
Phenotypes(Span("were readily detected from whole", sentence=459439, chars=[35,66], words=[5,9]), Span("AtCWINV4 transcripts", sentence=459439, chars=[14,33], words=[3,4]))
Phenotypes(Span("were readily detected from whole", sentence=459439, chars=[35,66], words=[5,9]), Span(

Phenotypes(Span("being covered by a highly reticulated", sentence=459521, chars=[150,186], words=[23,28]), Span("covered by a highly reticulated cuticle", sentence=459521, chars=[156,194], words=[24,29]))
Phenotypes(Span("including having multiple modified", sentence=459521, chars=[77,110], words=[11,14]), Span("structural hallmarks of wild-type nectaries", sentence=459521, chars=[32,74], words=[5,9]))
Phenotypes(Span("including having multiple modified", sentence=459521, chars=[77,110], words=[11,14]), Span("multiple modified stomata", sentence=459521, chars=[94,118], words=[13,15]))
Phenotypes(Span("including having multiple modified", sentence=459521, chars=[77,110], words=[11,14]), Span("covered by a highly reticulated cuticle", sentence=459521, chars=[156,194], words=[24,29]))
Phenotypes(Span("displayed all structural", sentence=459521, chars=[18,41], words=[3,5]), Span("structural hallmarks of wild-type nectaries", sentence=459521, chars=[32,74], words=[5,9]))
Phenotypes(Span("di

Phenotypes(Span("processes", sentence=439614, chars=[44,52], words=[9,9]), Span("as well as those depleted of the histone variant H2A.Z", sentence=439614, chars=[134,187], words=[26,35]))
Phenotypes(Span("defective in", sentence=439614, chars=[75,86], words=[14,15]), Span("as well as those depleted of the histone variant H2A.Z", sentence=439614, chars=[134,187], words=[26,35]))
Phenotypes(Span("variant", sentence=439614, chars=[175,181], words=[34,34]), Span("as well as those depleted of the histone variant H2A.Z", sentence=439614, chars=[134,187], words=[26,35]))
Phenotypes(Span("complex", sentence=439614, chars=[93,99], words=[17,17]), Span("as well as those depleted of the histone variant H2A.Z", sentence=439614, chars=[134,187], words=[26,35]))
Phenotypes(Span("severe growth defects in the", sentence=439641, chars=[4,31], words=[1,5]), Span("severe growth defects in the pie1 mutant", sentence=439641, chars=[4,43], words=[1,7]))
Phenotypes(Span("severe growth defects in the", senten

In [22]:
from snorkel.viewer import SentenceNgramViewer

sv = SentenceNgramViewer(candidates, session=session, n_per_page=6, height=400,
                         annotator_name='gold')
sv

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


<IPython.core.display.Javascript object>

In [ ]:
# Select spans from brat
# Select candidates from phenos
# For each candidate in phenos
# 